In [0]:
%run "../../FUNCOES/Classes_1.1"

In [0]:
### Monta data inicio e fim de extração da Bronze

days  = -3  ### determina D-x 
end   = date.today()
start = end + timedelta(days)
start_datetime = start.strftime("%Y%m%d")
end_datetime   = end.strftime("%Y%m%d")

In [0]:
###### Monta variaveis para leitura e gravação Delta Table
schema_bronze  = 'mawe_bronze'
bronze_table   = 'ADR6'
partition_key  = 'BK_EXTRACAO'
orderBy_key    = 'DATA_EXTRACAO'
from_to_field  = 'DATA_EXTRACAO'
target_table   = 'EMAIL_CLIENTES'

###### Leitura da tabela bronze 
df = Framework.read_bronze(schema_bronze, bronze_table, partition_key, orderBy_key, start_datetime, end_datetime, from_to_field)

In [0]:
delimiter     = '|'
extra_columns = ['BK_EXTRACAO', 'MES_EXTRACAO', 'ANO_EXTRACAO']

###### Busca Metadados, renomeia colunas e seleciona somente colunas que existem no dicionario 
df = Framework.read_metadata(df=df, file_format=Framework.RawFileFormat.CSV, table=target_table, delimiter=delimiter, extra_columns=extra_columns )

In [0]:
###### Busca dicionario de dados 
##df = Framework.decode_columns(df, target_table)


In [0]:
###### Renomeia as colunas do Data Frame
##df = Framework.rename_collumns(df)

In [0]:
###### Remove colunas que não existem no dicionario de dados 
##extra_columns = ['BK_EXTRACAO', 'MES_EXTRACAO', 'ANO_EXTRACAO']
##colunas = Framework.columns_to_keep(df, target_table, extra_columns) 
  

In [0]:
###### Mantem somente colunas finais para gravação da Silver  
##df = df.select(colunas)

In [0]:
###### Monta variaveis da Delta Table
schema_silver  = "mawe_silver"
silver_table = "EMAIL_CLIENTES"
key_columns  = ["BK_EXTRACAO"]
partition    = ["MES_EXTRACAO"]

###### busca caminho delta table na camada
location = Framework.generate_silver_table_location(
    schema_name=schema_silver, 
    table_name=silver_table,
)


## executa a gravação da tabela silver 
results = Framework.write_delta_table(
    df=df,
    location=location,
    schema_name=schema_silver,
    table_name=silver_table,
    load_type=Framework.LoadType.UPSERT,
    key_columns=key_columns,
    partition_columns=partition,
    schema_evolution_mode=Framework.SchemaEvolutionMode.ADD_NEW_COLUMNS,
)
